In [ ]:
# Import standard modules.
from importlib import import_module
import math as m
import os
import sys

# Import supplemental modules.
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
import numpy as np
import seaborn as sb

In [ ]:
# Specify the run ID (aka problem name).
runid = "loop2d_nBxByBz"

# Add the subdirectory for the run results to the module search path.
run_path = os.path.join(".", runid)
sys.path.append(run_path)

# Import the problem definition from the run results directory.
p = import_module(runid)

# Read the run hyperparameters from the run results directory.
import hyperparameters as hp

In [ ]:
# Define analytical solutions.

# Constants
Q = 60.0   # Flow angle in degrees clockwise from +y axis.
u0 = 1.0   # Initial flow speed.
A = 1      # Magnitude of magnetic vector potential.
R0 = 0.3   # Radius of current cylinder.

# Compute the constant velocity components.
u0x = u0*np.sin(np.radians(Q))
u0y = u0*np.cos(np.radians(Q))

# Constant values for other variables.
n0 = 1.0
B0z = 0.0

def n_analytical(t, x, y):
    n = np.full(t.shape, n0)
    return n

def Bx_analytical(t, x, y):
    r = np.sqrt((x - u0x*t)**2 + (y - u0y*t)**2)
    which = np.where(r < R0)
    Bx = np.zeros(r.shape)
    Bx[which] = -A*(y[which] - u0y*t[which])/r[which]
    return Bx

def By_analytical(t, x, y):
    r = np.sqrt((x - u0x*t)**2 + (y - u0y*t)**2)
    which = np.where(r < R0)
    By =np.zeros(r.shape)
    By[which] =  A*(x[which] - u0x*t[which])/r[which]
    return By

def Bz_analytical(t, x, y):
    Bz = np.full(t.shape, B0z)
    return Bz

In [ ]:
# Load all data.

# Load the training point coordinates.
txy_train = np.loadtxt(os.path.join(runid, "X_train.dat"))
t_train = txy_train[:, 0]
x_train = txy_train[:, 1]
y_train = txy_train[:, 2]

# Load the data locations and values (includes initial conditions).
txy_data = np.loadtxt(os.path.join(runid, "XY_data.dat"))

# Extract the initial conditions.
ic = txy_data[:, 3:]

# Load the model-predicted values.
ψ = []
delψ = []
for i in range(len(p.dependent_variable_names)):
    var_name = p.dependent_variable_names[i]
    ψ.append(np.loadtxt(os.path.join(runid, "%s_train.dat" % var_name)))
    delψ.append(np.loadtxt(os.path.join(runid, "del_%s_train.dat" % var_name)))

# Load the loss function histories.
losses_model = np.loadtxt(os.path.join(runid, "losses_model.dat"))
losses_model_res = np.loadtxt(os.path.join(runid, "losses_model_res.dat"))
losses_model_data = np.loadtxt(os.path.join(runid, "losses_model_data.dat"))
losses = np.loadtxt(os.path.join(runid, "losses.dat"))
losses_res = np.loadtxt(os.path.join(runid, "losses_res.dat"))
losses_data = np.loadtxt(os.path.join(runid, "losses_data.dat"))

In [ ]:
# Compute the limits of the training domain.
t_min = t_train[0]
t_max = t_train[-1]
x_min = x_train[0]
x_max = x_train[-1]
y_min = y_train[0]
y_max = y_train[-1]

# Extract the unique training point values (a grid is assumed).
t_train_vals = np.unique(t_train)
x_train_vals = np.unique(x_train)
y_train_vals = np.unique(y_train)
n_t_train_vals = len(t_train_vals)
n_x_train_vals = len(x_train_vals)
n_y_train_vals = len(y_train_vals)

In [ ]:
# Plotting options

# Compute the number of rows and columns for the 2-per-row plots.
n_cols = 2
n_rows = m.ceil(p.n_var/n_cols)

# Specify the size (inches) for individual subplots.
SUBPLOT_WIDTH = 5.0
SUBPLOT_HEIGHT = 5.0

# Compute the figure size for model loss plots.
model_loss_figsize = (SUBPLOT_WIDTH*n_cols, SUBPLOT_HEIGHT*n_rows)

# Compute the figure size for the total loss plot.
total_loss_figsize = (SUBPLOT_WIDTH*2, SUBPLOT_HEIGHT)

# Minimum and maximum values to show in loss plots.
L_min = 1e-6
L_max = 1

# Compute the figure size for side-by-side actual and predicted quiver plots for the magnetic field vectors.
B_vector_figsize = (SUBPLOT_WIDTH*2, SUBPLOT_HEIGHT)

# Compute the figure size for side-by-side actual, predicted, and error plots for the magnetic field magnitudes.
B_magnitude_figsize = (SUBPLOT_WIDTH*3, SUBPLOT_HEIGHT)

# Magnetic field intensity limits.
B_vmin = 0.9
B_vmax = 1.1

# N limits
n_vmin = 0.9
n_vmax = 1.1

# Bz limits.
Bz_vmin = -1e-6
Bz_vmax = 1e-6

# Absolute error heatmap limits.
abs_err_vmin = -1e-3
abs_err_vmax = 1e-3

# Compute the coordinate plot tick locations and labels.
XY_n_x_ticks = 5
XY_x_tick_pos = np.linspace(x_min, x_max, XY_n_x_ticks)
XY_x_tick_labels = ["%.2f" % x for x in XY_x_tick_pos]
XY_n_y_ticks = 5
XY_y_tick_pos = np.linspace(y_min, y_max, XY_n_y_ticks)
XY_y_tick_labels = ["%.2f" % y for y in XY_y_tick_pos]

# Compute the heat map tick locations and labels.
heatmap_n_x_ticks = 5
heatmap_x_tick_pos = np.linspace(0, n_x_train_vals - 1, heatmap_n_x_ticks)
heatmap_x_tick_labels = ["%.2f" % (x_min + x/(n_x_train_vals - 1)*(x_max - x_min)) for x in heatmap_x_tick_pos]
heatmap_n_y_ticks = 5
heatmap_y_tick_pos = np.linspace(0, n_y_train_vals - 1, heatmap_n_y_ticks)
heatmap_y_tick_labels = ["%.2f" % (y_min + y/(n_y_train_vals - 1)*(y_max - y_min)) for y in heatmap_y_tick_pos]
heatmap_y_tick_labels = list(reversed(heatmap_y_tick_labels))

In [ ]:
# Plot the loss history for each model.
fig = plt.figure(figsize=model_loss_figsize)
for i in range(p.n_var):
    ax = plt.subplot(n_rows, n_cols, i + 1)
    ax.semilogy(losses_model_res[:, i], label="$L_{res}$")
    ax.semilogy(losses_model_data[:, i], label="$L_{data}$")
    ax.semilogy(losses_model[:, i], label="$L$")
    ax.set_ylim(L_min, L_max)
    ax.set_xlabel("Epoch")
    if i % n_cols == 0:
        ax.set_ylabel("Loss function")
    ax.grid()
    ax.legend()
    ax.set_title(p.dependent_variable_labels[i])
fig.suptitle("Loss function histories by model")
plt.show()

In [ ]:
# Plot the total loss function history.
plt.figure(figsize=total_loss_figsize)
plt.semilogy(losses_res, label="$L_{res}$")
plt.semilogy(losses_data, label="$L_{data}$")
plt.semilogy(losses, label="$L$")
plt.ylim(L_min, L_max)
plt.xlabel("Epoch")
plt.ylabel("Loss function")
plt.grid()
plt.legend()
plt.title("Total loss function evolution for %s" % runid)
plt.show()

In [ ]:
# Plot the actual and predicted initial magnetic field vectors.
n_start = n_x_train_vals*n_y_train_vals
x = txy_data[:, 1]
y = txy_data[:, 2]
B0x_act = ic[:, 1]
B0y_act = ic[:, 2]
B0x_pred = ψ[1][:n_start]
B0y_pred = ψ[2][:n_start]

# Create the figure.
fig = plt.figure(figsize=B_vector_figsize)

# Actual
ax = plt.subplot(1, 2, 1)
ax.quiver(x, y, B0x_act, B0y_act)
ax.set_aspect(1.0)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(XY_x_tick_pos, XY_x_tick_labels)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(XY_y_tick_pos, XY_y_tick_labels)
ax.grid()
ax.set_title("Actual")

# Predicted
ax = plt.subplot(1, 2, 2)
ax.quiver(x, y, B0x_pred, B0y_pred)
ax.set_aspect(1.0)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(XY_x_tick_pos, XY_x_tick_labels)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(XY_y_tick_pos, XY_y_tick_labels)
ax.grid()
ax.set_title("Predicted")

fig.suptitle("Initial magnetic field")
plt.show()

In [ ]:
# Plot the actual, predicted, and errors in initial magnetic field magntiudes.
B0_act = np.sqrt(B0x_act**2 + B0y_act**2)
B0_pred = np.sqrt(B0x_pred**2 + B0y_pred**2)
B0_err = B0_pred - B0_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
B0_act_plot = np.flip(B0_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B0_act_plot, ax=ax, vmin=B_vmin, vmax=B_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Actual")

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
B0_pred_plot = np.flip(B0_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B0_pred_plot, ax=ax, vmin=B_vmin, vmax=B_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Predicted")

# Error
ax = plt.subplot(1, 3, 3)
B0_err_plot = np.flip(B0_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B0_err_plot, vmin=abs_err_vmin, vmax=abs_err_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Absolute error")

fig.suptitle("Initial magnetic field magnitude")
plt.show()

In [ ]:
# Plot the actual, predicted, and errors in initial n.
n0_act = ic[:, 0]
n0_pred = ψ[0][:n_start]
n0_err = n0_pred - n0_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
n0_act_plot = np.flip(n0_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(n0_act_plot, ax=ax, vmin=n_vmin, vmax=n_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Actual")

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
n0_pred_plot = np.flip(n0_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(n0_pred_plot, ax=ax, vmin=n_vmin, vmax=n_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Predicted")

# Error
ax = plt.subplot(1, 3, 3)
n0_err_plot = np.flip(n0_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(n0_err_plot, vmin=abs_err_vmin, vmax=abs_err_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Absolute error")

fig.suptitle("Initial %s" % p.dependent_variable_labels[0])
plt.show()

In [ ]:
# Plot the actual, predicted, and errors in initial Bz.
B0z_act = ic[:, 3]
B0z_pred = ψ[3][:n_start]
B0z_err = B0z_pred - B0z_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
B0z_act_plot = np.flip(B0z_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B0z_act_plot, ax=ax, vmin=Bz_vmin, vmax=Bz_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Actual")

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
B0z_pred_plot = np.flip(B0z_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B0z_pred_plot, ax=ax, vmin=Bz_vmin, vmax=Bz_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Predicted")

# Error
ax = plt.subplot(1, 3, 3)
B0z_err_plot = np.flip(B0z_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B0z_err_plot, vmin=abs_err_vmin, vmax=abs_err_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Absolute error")

fig.suptitle("Initial %s" % p.dependent_variable_labels[3])
plt.show()

In [ ]:
# Plot the actual and predicted final magnetic field vectors.
n_end = n_x_train_vals*n_y_train_vals
t = t_train[-n_end:]
x = x_train[-n_end:]
y = y_train[-n_end:]
B1x_act = Bx_analytical(t, x, y)
B1y_act = By_analytical(t, x, y)
B1x_pred = ψ[1][-n_end:]
B1y_pred = ψ[2][-n_end:]

# Create the figure.
fig = plt.figure(figsize=B_vector_figsize)

# Actual
ax = plt.subplot(1, 2, 1)
ax.quiver(x, y, B1x_act, B1y_act)
ax.set_aspect(1.0)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(XY_x_tick_pos, XY_x_tick_labels)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(XY_y_tick_pos, XY_y_tick_labels)
ax.grid()
ax.set_title("Actual")

# Predicted
ax = plt.subplot(1, 2, 2)
ax.quiver(x, y, B1x_pred, B1y_pred)
ax.set_aspect(1.0)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(XY_x_tick_pos, XY_x_tick_labels)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(XY_y_tick_pos, XY_y_tick_labels)
ax.grid()
ax.set_title("Predicted")

fig.suptitle("Final magnetic field")
plt.show()

In [ ]:
# Plot the actual, predicted, and errors in final magnetic field magntiudes.
B1_act = np.sqrt(B1x_act**2 + B1y_act**2)
B1_pred = np.sqrt(B1x_pred**2 + B1y_pred**2)
B1_err = B1_pred - B1_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
B1_act_plot = np.flip(B1_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B1_act_plot, ax=ax, vmin=B_vmin, vmax=B_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Actual")

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
B1_pred_plot = np.flip(B1_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B1_pred_plot, ax=ax, vmin=B_vmin, vmax=B_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Predicted")

# Error
ax = plt.subplot(1, 3, 3)
B1_err_plot = np.flip(B1_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B1_err_plot, vmin=abs_err_vmin, vmax=abs_err_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Absolute error")

fig.suptitle("Final magnetic field magnitude")
plt.show()

In [ ]:
# Plot the actual, predicted, and errors in final n.
n1_act = Bz_analytical(t, x, y)
n1_pred = ψ[0][-n_end:]
n1_err = n1_pred - n1_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
n1_act_plot = np.flip(n1_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(n1_act_plot, ax=ax, vmin=n_vmin, vmax=n_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Actual")

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
n1_pred_plot = np.flip(n1_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(n1_pred_plot, ax=ax, vmin=n_vmin, vmax=n_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Predicted")

# Error
ax = plt.subplot(1, 3, 3)
n1_err_plot = np.flip(n1_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(n1_err_plot, vmin=abs_err_vmin, vmax=abs_err_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Absolute error")

fig.suptitle("Final %s" % p.dependent_variable_labels[0])
plt.show()

In [ ]:
# Plot the actual, predicted, and errors in final Bz.
B1z_act = Bz_analytical(t, x, y)
B1z_pred = ψ[3][-n_end:]
B1z_err = B1z_pred - B1z_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
B1z_act_plot = np.flip(B1z_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B1z_act_plot, ax=ax, vmin=Bz_vmin, vmax=Bz_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Actual")

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
B1z_pred_plot = np.flip(B1z_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B1z_pred_plot, ax=ax, vmin=Bz_vmin, vmax=Bz_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Predicted")

# Error
ax = plt.subplot(1, 3, 3)
B1z_err_plot = np.flip(B1z_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B1z_err_plot, vmin=abs_err_vmin, vmax=abs_err_vmax, square=True)
ax.set_xlabel(p.independent_variable_labels[1])
ax.set_xticks(heatmap_x_tick_pos, heatmap_x_tick_labels, rotation=0)
ax.set_ylabel(p.independent_variable_labels[2])
ax.set_yticks(heatmap_y_tick_pos, heatmap_y_tick_labels)
ax.grid()
ax.set_title("Absolute error")

fig.suptitle("Final %s" % p.dependent_variable_labels[3])
plt.show()